# Bond Option Comparison with DerivaGem Function 17

Value an option on a coupon paying bond using the Hull-White in response to a question.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
from financepy.utils import *
from financepy.market.curves import *
from financepy.models.hw_tree import HWTree
from financepy.products.bonds import *

## Set up the Discount Curve

In [3]:
settle_dt = Date(1, 12, 2019)

Set up discount curve

In [4]:
rate = 0.05
dc_type = DayCountTypes.THIRTY_360_BOND
fixed_freq = FrequencyTypes.SEMI_ANNUAL
discount_curve = DiscountCurveFlat(settle_dt, rate, fixed_freq, dc_type)

In [5]:
df_times = discount_curve.times
df_values = discount_curve.dfs

In [6]:
df_values

array([1.        , 0.9877296 , 0.97560976, 0.96363863, 0.9518144 ,
       0.94013525, 0.92859941, 0.91720512, 0.90595064, 0.89483426,
       0.88385429, 0.87300904, 0.86229687, 0.85171614, 0.84126524,
       0.83094257, 0.82074657, 0.81067568, 0.80072836, 0.7909031 ,
       0.7811984 , 0.77161278, 0.76214478, 0.75279296, 0.74355589,
       0.73443215, 0.72542038, 0.71651918, 0.7077272 , 0.6990431 ,
       0.69046556, 0.68199327, 0.67362493, 0.66535928, 0.65719506,
       0.64913101, 0.64116591, 0.63329855, 0.62552772, 0.61785224,
       0.61027094])

## Set up the Bond Option

First create the bond

In [7]:
issue_dt = Date(1,12,2018)
maturity_dt = issue_dt.add_tenor("10Y")
coupon = 0.05
freq_type = FrequencyTypes.SEMI_ANNUAL
dc_type = DayCountTypes.THIRTY_360_BOND
bond = Bond(issue_dt, maturity_dt, coupon, freq_type, dc_type)

Let's first price the bond on the libor curve

In [8]:
cp = bond.clean_price_from_discount_curve(settle_dt, discount_curve)
dp = bond.dirty_price_from_discount_curve(settle_dt, discount_curve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Dirty Price: %9.3f"% dp)

Fixed Income Clean Price:   100.000
Fixed Income Dirty Price:   100.000


In [9]:
settle_dt

01-DEC-2019

In [10]:
bond.print_payments(settle_dt)

 01-JUN-2020      2.50000 
 01-DEC-2020      2.50000 
 01-JUN-2021      2.50000 
 01-DEC-2021      2.50000 
 01-JUN-2022      2.50000 
 01-DEC-2022      2.50000 
 01-JUN-2023      2.50000 
 01-DEC-2023      2.50000 
 01-JUN-2024      2.50000 
 01-DEC-2024      2.50000 
 01-JUN-2025      2.50000 
 01-DEC-2025      2.50000 
 01-JUN-2026      2.50000 
 01-DEC-2026      2.50000 
 01-JUN-2027      2.50000 
 01-DEC-2027      2.50000 
 01-JUN-2028      2.50000 
 01-DEC-2028    102.50000 



Then define and create the option

In [11]:
expiry_dt = settle_dt.add_tenor("18m")
strike_price = 100.0
face_amount = 100.0

In [12]:
europeanCallBondOption = BondOption(bond, expiry_dt, strike_price, OptionTypes.EUROPEAN_CALL)
americanCallBondOption = BondOption(bond, expiry_dt, strike_price, OptionTypes.AMERICAN_CALL)

In [13]:
europeanPutBondOption = BondOption(bond, expiry_dt, strike_price, OptionTypes.EUROPEAN_PUT)
americanPutBondOption = BondOption(bond, expiry_dt, strike_price, OptionTypes.AMERICAN_PUT)

Consider the forward bond price.

In [14]:
cp = bond.clean_price_from_discount_curve(expiry_dt, discount_curve)
dp = bond.dirty_price_from_discount_curve(expiry_dt, discount_curve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Dirty Price: %9.3f"% dp)

Fixed Income Clean Price:   100.000
Fixed Income Dirty Price:   100.000


Set the model parameters, start with zero vol.

In [15]:
num_steps = 500

In [16]:
sigma = 0.0125
a = 0.1
modelHW = HWTree(sigma, a, num_steps)

In [17]:
ec = europeanCallBondOption.value(settle_dt, discount_curve, modelHW)
ac = americanCallBondOption.value(settle_dt, discount_curve, modelHW)
print("European Call Value: %9.5f" % ec)
print("American Call Value: %9.5f" % ac)

European Call Value:   2.39763
American Call Value:   2.66598


In [18]:
ec = europeanPutBondOption.value(settle_dt, discount_curve, modelHW)
ac = americanPutBondOption.value(settle_dt, discount_curve, modelHW)
print("European Put Value: %9.5f" % ec)
print("American Put Value: %9.5f" % ac)

European Put Value:   2.42253
American Put Value:   2.68292


# COMPARISON

In [19]:
if 1==1:

    settle_dt = Date(1, 12, 2019)
    issue_dt = Date(1, 12, 2018)
    expiry_dt = settle_dt.add_tenor("18m")
    maturity_dt = settle_dt.add_tenor("10Y")
    coupon = 0.05
    freq_type = FrequencyTypes.SEMI_ANNUAL
    accrual_type = DayCountTypes.THIRTY_360_BOND
    bond = Bond(issue_dt, maturity_dt, coupon, freq_type,  accrual_type)

    couponTimes = []
    couponFlows = []
    cpn = bond.cpn/bond.freq

    numFlows = len(bond.cpn_dts)
    for i in range(0, numFlows):

        pcd = bond.cpn_dts[i-1]
        ncd = bond.cpn_dts[i]

        if ncd > settle_dt:

            if len(couponTimes) == 0:
                flowTime = (pcd - settle_dt) / G_DAYS_IN_YEARS
                couponTimes.append(flowTime)
                couponFlows.append(cpn)

            flowTime = (ncd - settle_dt) / G_DAYS_IN_YEARS
            couponTimes.append(flowTime)
            couponFlows.append(cpn)

    couponTimes = np.array(couponTimes)
    couponFlows = np.array(couponFlows)

    strikePrice = 100.0
    face = 100.0

    model = HWTree(sigma, a, None)

    #  Test convergence
    t_exp = (expiry_dt - settle_dt)/G_DAYS_IN_YEARS
    t_mat = (maturity_dt - settle_dt)/G_DAYS_IN_YEARS

    # Jamshidian approach
    v_jam = model.european_bond_option_jamshidian(t_exp, strikePrice, face,
                                              couponTimes, couponFlows,
                                              df_times, df_values)

    model.num_time_steps = 200
    model.build_tree(t_mat, df_times, df_values)
    exerciseType = FinExerciseTypes.EUROPEAN
    vHW = model.bond_option(t_exp, strikePrice, face,
                            couponTimes, couponFlows, exerciseType)


In [20]:
v_jam

{'call': np.float64(2.557167935570238), 'put': np.float64(2.582591550479875)}

In [21]:
vHW

{'call': 2.5651092396164996, 'put': 2.588240081574218}

Copyright (c) Dominic O'Kane 2021